In [ ]:
from razdel import sentenize, tokenize
import pymorphy3
import re
import pandas as pd
from collections import Counter, defaultdict
import matplotlib.pyplot as plt
import seaborn as sns

with open("/content/DOVLATOV_SOBR_SOCH.txt", "r", encoding="utf-8") as f:
    text = f.read()

sentences = [s.text for s in sentenize(text)]

morph = pymorphy3.MorphAnalyzer()

alcohol_words = ['выпил', 'выпить', 'водка', 'пьяный', 'опохмел', 'запой', 'бутылка']
alcohol_lemmas = {morph.parse(w)[0].normal_form for w in alcohol_words}

percept_verbs_raw = ['увидеть', 'почувствовать', 'услышать', 'ощутить', 'заметить', 'различить', 'ощущать']
percept_lemmas = {morph.parse(w)[0].normal_form for w in percept_verbs_raw}

visual_words = {
    'размытый', 'расплывчатый', 'туманный', 'мутный', 'смазанный',
    'нежный', 'мягкий', 'тёплый', 'золотистый', 'сияющий',
    'чёрный', 'серый', 'мрачный', 'яркий', 'светлый'
}
auditory_words = {
    'гул', 'шум', 'звон', 'жужжание', 'мерный', 'тихий', 'громкий',
    'приглушённый', 'далёкий', 'резкий', 'монотонный'
}
emotional_words = {
    'гармония', 'примирение', 'спокойствие', 'тоска', 'радость',
    'грусть', 'тревога', 'облегчение', 'восторг', 'апатия',
    'умиротворение', 'раздражение', 'любовь', 'ненависть'
}

visual_lemmas = {morph.parse(w)[0].normal_form for w in visual_words}
auditory_lemmas = {morph.parse(w)[0].normal_form for w in auditory_words}
emotional_lemmas = {morph.parse(w)[0].normal_form for w in emotional_words}

emotional_lemmas через normal_form
emotional_lemmas = {morph.parse(w)[0].normal_form for w in emotional_words}

def get_context_windows(sentences, condition_func, window=3):
    contexts = []
    for i, sent in enumerate(sentences):
        if condition_func(sent):
            before = sentences[max(0, i - window):i]
            after = sentences[i + 1:i + 1 + window]
            contexts.append((sent, " ".join(before), " ".join(after)))
    return contexts

def is_alcohol_sentence(sent):
    words = [t.text.lower() for t in tokenize(sent) if t.text.isalpha()]
    lemmas = [morph.parse(w)[0].normal_form for w in words]
    return any(l in alcohol_lemmas for l in lemmas)

alcohol_episodes = get_context_windows(sentences, is_alcohol_sentence, window=3)

print(f" Найдено {len(alcohol_episodes)} эпизодов с алкоголем.")

def extract_percept_descriptions(context_text):
   tokens = [t.text.lower() for t in tokenize(context_text) if t.text.isalpha()]
   lemmas = [morph.parse(w)[0].normal_form for w in tokens]

    percept_positions = [i for i, l in enumerate(lemmas) if l in percept_lemmas]

    descriptions = {'visual': [], 'auditory': [], 'emotional': []}

        for pos in percept_positions:
        start = max(0, pos - 5)
        end = min(len(lemmas), pos + 6)
        window_lemmas = lemmas[start:end]

        for l in window_lemmas:
            if l in visual_lemmas:
                descriptions['visual'].append(l)
            elif l in auditory_lemmas:
                descriptions['auditory'].append(l)
            elif l in emotional_lemmas:
                descriptions['emotional'].append(l)

    return descriptions

all_descriptions_after = {'visual': [], 'auditory': [], 'emotional': []}
all_descriptions_before = {'visual': [], 'auditory': [], 'emotional': []}

for _, before_ctx, after_ctx in alcohol_episodes:
    desc_before = extract_percept_descriptions(before_ctx)
    desc_after = extract_percept_descriptions(after_ctx)

    for key in all_descriptions_after:
        all_descriptions_after[key].extend(desc_after[key])
        all_descriptions_before[key].extend(desc_before[key])

def compare_frequencies(before, after, top_n=10):
    cnt_before = Counter(before)
    cnt_after = Counter(after)
    all_words = set(cnt_before.keys()) | set(cnt_after.keys())

    if not all_words:
        return pd.DataFrame()   DataFrame

    data = []
    for word in all_words:
        data.append({
            'Слово': word,
            'До': cnt_before[word],
            'После': cnt_after[word],
            'Разница (После – До)': cnt_after[word] - cnt_before[word]
        })

    df = pd.DataFrame(data)
    df = df.sort_values('Разница (После – До)', ascending=False)
    return df.head(top_n)

categories = ['visual', 'auditory', 'emotional']
titles = {'visual': 'Визуальное восприятие', 'auditory': 'Аудиальное восприятие', 'emotional': 'Эмоциональное состояние'}

plt.figure(figsize=(15, 5))
for i, cat in enumerate(categories):
    plt.subplot(1, 3, i + 1)
    before = all_descriptions_before[cat]
    after = all_descriptions_after[cat]


    common_words = set(Counter(before).most_common(8)) | set(Counter(after).most_common(8))
    words = list({w for w, _ in common_words})

    if not words:
        plt.title(titles[cat] + "\n(нет данных)")
        continue

    freq_before = [before.count(w) for w in words]
    freq_after = [after.count(w) for w in words]

    x = range(len(words))
    plt.bar([xi - 0.2 for xi in x], freq_before, width=0.4, label='До', color='lightblue')
    plt.bar([xi + 0.2 for xi in x], freq_after, width=0.4, label='После', color='lightcoral')
    plt.xticks(x, words, rotation=45, ha='right')
    plt.title(titles[cat])
    plt.legend()

plt.tight_layout()
plt.show()

print("Топ-10 изменений в восприятии ПОСЛЕ употребления алкоголя:")

for cat in categories:
    before_list = all_descriptions_before[cat]
    after_list = all_descriptions_after[cat]


    if not before_list and not after_list:
        print(f"\n🔹 {titles[cat]}: нет перцептивных описаний")
        continue

    df_diff = compare_frequencies(before_list, after_list)


    if df_diff.empty:
        print(f"\n🔹 {titles[cat]}: нет значимых слов для сравнения")
        continue

    print(f"\n🔹 {titles[cat]}:")

    cols_to_show = ['Слово', 'До', 'После']
    if 'Разница (После – До)' in df_diff.columns:
        cols_to_show.append('Разница (После – До)')
    display(df_diff[cols_to_show].reset_index(drop=True))
